## Accuracy Linear FOM

Environment

In [1]:
import sys
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=8,
  floatx="float64"
)

Import libraries

In [2]:
import numpy as np
import scipy as sp
import joblib as jl

from tqdm import tqdm
from ronek import utils
from ronek import postproc as pp
from ronek.systems import TASystem

In [3]:
import matplotlib as mpl
# Use custom style
# See: https://matplotlib.org/1.5.3/users/style_sheets.html
path_to_styles = "/home/zanardi/Workspace/Research/styles/matplotlib/"
style_name = "paper_1column"
mpl.style.use(f"{path_to_styles}/{style_name}.mplstyle")

Define inputs

In [4]:
T = 1e4
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/RVC_O3/database/"
path_to_data = "/home/zanardi/Codes/ML/RONEK/run/RVC_O3/test/data/"
nb_samples = 1000
nb_workers = 10

Initialize isothermal master equation model

In [5]:
system = TASystem(
  T=T,
  rates=path_to_dtb + "/kinetics.hdf5",
  species={
    k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  use_einsum=False,
  use_factorial=True
)

Solve linear FOM

In [6]:
def compute_linfom_err(path, index):
  # Load data
  data = utils.load_case(path=path, index=index)
  n0, t, n = [data[k] for k in ("n0", "t", "n")]
  rho = data["mu"][-1]
  # Solve linear system
  nl = system.solve_lin_fom(t, n0)
  # Convert to mass fractions
  w = system.mix.get_w(np.vstack(n), rho)
  wl = system.mix.get_w(np.vstack(nl), rho)
  # Compute error
  err = utils.l2_relative_error(w, wl, axis=0)
  return t, err

In [ ]:
err = jl.Parallel(nb_workers)(
  jl.delayed(compute_linfom_err)(
    path=path_to_data,
    index=i
  ) for i in tqdm(
    iterable=range(nb_samples),
    ncols=80,
    file=sys.stdout
  )
)
t, err = list(zip(*err))
err = np.vstack(err)

Plot relative $L^2$ error

In [9]:
pp.plot_err_ci_evolution(
  x=t[0],
  mean=np.mean(err, axis=0),
  sem=sp.stats.sem(err, axis=0),
  size=len(err),
  alpha=0.95,
  xlim=None,
  ylim=None,
  hline=None,
  labels=[r"$t$ [s]", r"Relative $L^2$ Error"],
  scales=["log", "linear"],
  legend_loc="best",
  figname="./rel_err_lin_fom.pdf",
  save=True,
  show=False
)